In [74]:
%reload_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
import numpy as np
import h5py
import sys,time,json
import random
from IPython.display import display
import fidle
from myutils.dataset import  read_dataset,get_data_generator_v1,get_data_generator_v2
from myutils.models import get_model_v1,get_model_v2,get_model_v3
from myutils.experimental_protocol import experimental_protocol_multi_run
import fidle
import pandas as pd

In [75]:
enhanced_dir = './data-1-scale'

# Protocol experimental
# datasets      = ['set-24x24-L', 'set-24x24-RGB']
datasets = ['set-24x24-L-LHE','set-24x24-RGB','set-48x48-L','set-48x48-RGB','set-48x48-RGB-HE']
models        = ['get_model_v1','get_model_v2', 'get_model_v3']
data_generators = [None,'get_data_generator_v1','get_data_generator_v2']
batch_size    = 64
epochs        = 20
scale         = 1

fit_verbosity = 0
run_dir ='./proto-exp-{}-scale-out'.format(scale)

In [76]:
random.seed(time.time())

# ---- Where I am ?
now    = time.strftime("%A %d %B %Y - %Hh%Mm%Ss")
here   = os.getcwd()
tag_id = '{:06}'.format(random.randint(0,99999))


fidle.utils.subtitle('Full Convolution Notebook')
print('Now is             : ', now      )
print('Tag id             : ', tag_id   )
print('Working directory  : ', here     )
print('Output  directory  : ', run_dir  )

<br>**Full Convolution Notebook**

Now is             :  Wednesday 27 December 2023 - 21h42m06s
Tag id             :  079698
Working directory  :  /home/20015587/Documents/ING3/DeepL/TP1/resolution
Output  directory  :  ./proto-exp-1-scale-out


In [77]:
chrono=fidle.Chrono()
chrono.start()

print('\n---- Run','-'*30)

csv_report_output,csv_metadtat_output = experimental_protocol_multi_run(enhanced_dir,
                                                                        run_dir,
                                                                        datasets,
                                                                        models,
                                                                        data_generators,
                                                                        scale,
                                                                        batch_size,
                                                                        epochs,
                                                                        fit_verbosity,
                                                                        tag_id,
                                                                        )


# ---- Save report

report_name=f'{run_dir}/report_{tag_id}.csv'
metadata_name=f'{run_dir}/metadata_{tag_id}.csv'

with open(report_name, 'w') as file:
    file.write(csv_report_output)

with open(metadata_name,'w') as file:
    file.write(csv_metadtat_output)

print('\nReport saved as ',report_name)
print('\nMetadata saved as ', metadata_name)

chrono.show()
print('-'*50)



---- Run ------------------------------


<br>**Dataset : set-24x24-L-LHE**

Datasets have been shuffled.
Datasets have been resized with a factor  1
    Run model model_v1  : 
          With Datagenerator: False
Accuracy=  68.86    Duration=  59.21
          With Datagenerator: True


2023-12-27 21:43:05.882264: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Accuracy=  72.16    Duration=  60.69
          With Datagenerator: True


...

Accuracy=  76.05    Duration= 189.26
    Run model model_v3  : 
          With Datagenerator: False
Accuracy=  63.47    Duration= 207.67
          With Datagenerator: True


2023-12-27 23:40:20.938975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Accuracy=  72.75    Duration= 204.86
          With Datagenerator: True


2023-12-27 23:43:45.804873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Accuracy=  73.35    Duration= 202.26

Report saved as  ./proto-exp-1-scale-out/report_079698.csv

Metadata saved as  ./proto-exp-1-scale-out/metadata_079698.csv
Duration :  7501.41 seconds
--------------------------------------------------


In [78]:
report = pd.read_csv(report_name)
report.head(100)

,Dataset,DatasetSize,Model,Datagen,Duration,Accuracy
0,set-24x24-L-LHE,7.341827,model_v1,none,59.205537,68.862277
1,set-24x24-L-LHE,7.341827,model_v1,generator_v1,60.687897,72.155690
2,set-24x24-L-LHE,7.341827,model_v1,generator_v2,1227.575542,72.455090
3,set-24x24-L-LHE,7.341827,model_v2,none,68.514913,70.359284
4,set-24x24-L-LHE,7.341827,model_v2,generator_v1,72.608461,73.652697
5,set-24x24-L-LHE,7.341827,model_v2,generator_v2,76.406427,71.856290
6,set-24x24-L-LHE,7.341827,model_v3,none,88.987368,64.670658
7,set-24x24-L-LHE,7.341827,model_v3,generator_v1,86.990403,68.562877
8,set-24x24-L-LHE,7.341827,model_v3,generator_v2,81.306295,70.359284
9,set-24x24-RGB,21.984406,model_v1,none,57.100912,73.952097


In [ ]:
report[report["Accuracy"] >= report["Accuracy"].max()]